<a href="https://colab.research.google.com/github/patricksabry1/42028-Deep-Learning/blob/master/42028_Assignment_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MNIST Classification

## Load the dataset and store into train/test split variables

In [0]:
import tensorflow as tf
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()